# Generating Counterfactuals

In this notebook, we will focus on generating counterfactuals from individual
datapoints. This will be implemented for the following models:

- Naive Bayes
- Fair Bayesian Network
- Fair Random Forest Classifier

By generating counterfactuals, we hope to gain insight into how the model uses
the different attributes in it's decisions.

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.bayesnet import latentLabelClassifier, interpretableNaiveBayes
import pandas as pd
from random import sample
import numpy as np
from forseti.datproc import translate_categorical
from forseti.tree import interpretableTree

df = pd.read_csv("data/adult.csv")

clf = interpretableNaiveBayes()

tmp = df.copy(deep=True)
label = "income"
clf.train(label, df, "NB")
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)

## Naive Bayes, Black Female Low Income

In [3]:
idx = df[
    (df.gender == 'Female') & # Female
    (df.race == 'Black') & # Black
    (df.income == '<=50K') # Low Income
].sample(1).index[0]

idx = 23356

datapoint = pd.DataFrame(tmp.loc[idx]).T
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,
/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:472: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi.values.sum()
/home/bcwein/Code/Forseti/forseti/bayesnet.py:523: RuntimeWarning: divide by zero encountered in double_scalars
  abs(In[obj].max() - In[obj].min())
/home/bcwein/Code/Forseti/forseti/bayesnet.py:523: RuntimeWarning: invalid value encountered in double_scalars
  abs(In[obj].max() - In[obj].min())


## Show the Datapoint

In [4]:
print(
    datapoint.to_latex()
)

\begin{tabular}{lllllllllll}
\toprule
{} &             age & workclass & education & marital-status & occupation & relationship &   race &  gender &          capital-gain & hours-per-week \\
\midrule
23356 &  (16.927, 31.6] &         ? &   HS-grad &      Separated &          ? &    Unmarried &  Black &  Female &  (-4460.355, 16515.0] &   (20.6, 40.2] \\
\bottomrule
\end{tabular}



## Show Counterfactuals

In [5]:
print(
    R[
        (R['O1'] <= 0.5)
    ].sort_values(['O1', 'O3']).to_latex()
)

\begin{tabular}{lllllllllllrrrr}
\toprule
Empty DataFrame
Columns: Index(['age', 'workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'gender', 'capital-gain', 'hours-per-week',
       'O1', 'O2', 'O3', 'O4'],
      dtype='object')
Index: Int64Index([], dtype='int64') \\
\bottomrule
\end{tabular}



## Naive Bayes Without Sensitive, Black Female Low Income

In [16]:
df = pd.read_csv("data/adult.csv")

clf = interpretableNaiveBayes()

tmp = df.copy(deep=True)
label = "income"
df[['race', 'gender']] = np.random.permutation(df[['race', 'gender']])
clf.train(label, df, "NB")
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)

datapoint = pd.DataFrame(tmp.loc[idx]).T
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,
/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:472: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi.values.sum()


## Show the datapoint

In [17]:
print(datapoint.to_latex())

\begin{tabular}{lllllllllll}
\toprule
{} &             age & workclass & education & marital-status & occupation & relationship &   race &  gender &          capital-gain & hours-per-week \\
\midrule
23356 &  (16.927, 31.6] &         ? &   HS-grad &      Separated &          ? &    Unmarried &  Black &  Female &  (-4460.355, 16515.0] &   (20.6, 40.2] \\
\bottomrule
\end{tabular}



## Show the counterfactuals

In [18]:
print(
    R[
        (R['O1'] <= 0.5)
    ].sort_values(['O1', 'O3']).to_latex()
)

\begin{tabular}{lllllllllllrrrr}
\toprule
{} &             age &  workclass &  education &     marital-status & occupation &   relationship &                race &  gender &          capital-gain & hours-per-week &        O1 &   O2 &  O3 &   O4 \\
\midrule
128 &  (16.927, 31.6] &  State-gov &    HS-grad &          Separated &          ? &      Unmarried &               Black &    Male &    (79128.0, 99999.0] &   (20.6, 40.2] &  0.000000 &  0.7 &   3 &  0.0 \\
97  &    (31.6, 46.2] &          ? &    HS-grad &          Separated &          ? &  Not-in-family &               Black &    Male &    (79128.0, 99999.0] &   (20.6, 40.2] &  0.000000 &  0.6 &   4 &  0.0 \\
147 &  (16.927, 31.6] &  State-gov &  Doctorate &          Separated &          ? &      Unmarried &               Black &    Male &    (79128.0, 99999.0] &   (20.6, 40.2] &  0.000000 &  0.6 &   4 &  0.0 \\
136 &    (31.6, 46.2] &  State-gov &  Doctorate &  Married-AF-spouse &          ? &        Husband &  Amer-Indian-Eskimo &

## Fair Bayesian Network

In [9]:
df = pd.read_csv("data/adult.csv")
tmp = df.copy(deep=True)
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)
label = "income"
sensitives = ['gender', 'race']

clf = latentLabelClassifier(
    df, 
    sensitives, 
    label
)

datapoint = pd.DataFrame(tmp.iloc[idx]).T
clf.load('trained-models/fair_model.sav')
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

## Show the datapoint

In [10]:
print(datapoint.to_latex())

\begin{tabular}{lllllllllll}
\toprule
{} &             age & workclass & education & marital-status & occupation & relationship &   race &  gender &          capital-gain & hours-per-week \\
\midrule
23356 &  (16.927, 31.6] &         ? &   HS-grad &      Separated &          ? &    Unmarried &  Black &  Female &  (-4460.355, 16515.0] &   (20.6, 40.2] \\
\bottomrule
\end{tabular}



## Show the counterfactuals

In [11]:
print(
    R[
        (R['O1'] <= 0.5)
    ].sort_values(['O1', 'O3']).to_latex()
)

\begin{tabular}{lllllllllllrrrr}
\toprule
{} &             age &         workclass &   education & marital-status &       occupation &   relationship &                race &  gender &        capital-gain & hours-per-week &            O1 &   O2 &  O3 &   O4 \\
\midrule
111 &  (16.927, 31.6] &                 ? &        11th &      Separated &                ? &      Unmarried &               Black &  Female &  (79128.0, 99999.0] &   (79.4, 99.0] &  0.000000e+00 &  0.7 &   3 &  0.0 \\
114 &  (16.927, 31.6] &                 ? &     HS-grad &      Separated &                ? &      Unmarried &               White &  Female &  (79128.0, 99999.0] &   (79.4, 99.0] &  0.000000e+00 &  0.7 &   3 &  0.0 \\
162 &  (16.927, 31.6] &                 ? &        11th &      Separated &                ? &      Unmarried &               White &  Female &  (79128.0, 99999.0] &   (20.6, 40.2] &  0.000000e+00 &  0.7 &   3 &  0.1 \\
58  &  (16.927, 31.6] &      Never-worked &     HS-grad &      Separated &